# 1. 환경 설정, 모델 구조 정의 및 가중치 로드

In [1]:
# inference.ipynb - CELL 1

# ----------------- 1. 환경 설정 및 라이브러리 임포트 -----------------
!pip install transformers torch seqeval pytorch-crf tqdm sentencepiece tokenizers > /dev/null

import os
import torch
from torch import nn
from torchcrf import CRF
from transformers import AutoTokenizer, AutoModel, BertModel
from seqeval.metrics import f1_score # F1 Score는 사용하지 않지만, 임포트 환경을 유지합니다.
from google.colab import drive
import numpy as np

# ----------------- 2. 상수 및 경로 정의 (evaluation.ipynb와 동일) -----------------
KOBERT_MODEL_NAME = "skt/kobert-base-v1"
labels = ['O', 'B-SUB', 'I-SUB', 'B-DT', 'I-DT', 'B-LOC', 'I-LOC']
label_to_id = {label: i for i, label in enumerate(labels)}
id_to_label = {i: label for i, label in enumerate(labels)}
NUM_LABELS = len(labels)
MAX_LEN = 64

# 👇 Google Drive 마운트 및 경로 설정
if not os.path.exists('/content/drive'):
    print("🔄 Google Drive를 마운트합니다...")
    drive.mount('/content/drive')
else:
    print("✅ Google Drive가 이미 마운트되어 있습니다.")

# 🚨 모델 가중치 파일의 정확한 Google Drive 경로 (이전 단계에서 확인됨)
ML_ASSIGNMENT5_PATH = '/content/drive/MyDrive/ML_assignment5/'
BEST_MODEL_PATH = os.path.join(ML_ASSIGNMENT5_PATH, 'best_kobert_crf_model.pt')
# ----------------------------------------------------

# ----------------- 3. 모델 클래스 정의 (training.ipynb와 동일) -----------------
class KoBERTNerModel(nn.Module):
    def __init__(self, num_labels):
        super(KoBERTNerModel, self).__init__()
        self.bert = AutoModel.from_pretrained(KOBERT_MODEL_NAME)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]

        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        if labels is not None:
            mask = attention_mask.bool()
            loss = -self.crf(logits, labels, mask=mask, reduction='mean')
            return loss, logits
        else:
            mask = attention_mask.bool()
            tags = self.crf.decode(logits, mask=mask)
            return tags, logits

# ----------------- 4. 모델 로드 및 설정 -----------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = KoBERTNerModel(NUM_LABELS).to(device)

try:
    model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device))
    model.eval() # 추론 모드로 설정
    print(f"\n✅ 모델 가중치 로드 및 추론 모드 설정 성공: {BEST_MODEL_PATH}")
    tokenizer = AutoTokenizer.from_pretrained(KOBERT_MODEL_NAME)
except Exception as e:
    print(f"\n❌ 오류: 모델 가중치 로드 실패. 경로 확인 필요.")
    print(f"상세 오류: {e}")

🔄 Google Drive를 마운트합니다...
Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]


✅ 모델 가중치 로드 및 추론 모드 설정 성공: /content/drive/MyDrive/ML_assignment5/best_kobert_crf_model.pt


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

# 2. 추론 함수 정의 및 결과 재구성

In [2]:
# inference.ipynb - CELL 2

# [핵심] BIO 태그를 캘린더 정보로 재구성하는 함수
def parse_bio_to_schedule(tokens, pred_tags):
    schedule = {
        "제목 (SUB)": [],
        "날짜/시간 (DT)": [],
        "장소 (LOC)": []
    }

    current_entity = {"tag": None, "tokens": []}

    # [CLS]와 [SEP] 토큰을 제외하고 순회 (1번 인덱스부터 시작)
    # pred_tags는 List[int] 형태이므로, id_to_label을 통해 문자열 태그로 변환해야 함

    # tokens의 첫 번째와 마지막은 [CLS], [SEP]일 확률이 높으므로 유효 토큰만 순회
    valid_tokens = tokens[1:-1]
    valid_tags = [id_to_label[tag] for tag in pred_tags[0]][1:-1]

    for token, tag in zip(valid_tokens, valid_tags):

        # 1. 새 엔티티 시작 (B-태그)
        if tag.startswith('B-'):
            if current_entity["tag"] is not None:
                # 이전 엔티티 저장
                schedule[current_entity["tag"]].append("".join(current_entity["tokens"]).replace(' ', ''))

            # 새 엔티티 초기화
            current_entity["tag"] = tag[2:] # B-SUB -> SUB
            current_entity["tokens"] = [token.replace(' ', ' ')] # KoBERT의 Josa 토큰화 방식 고려

        # 2. 엔티티 내부 (I-태그)
        elif tag.startswith('I-') and current_entity["tag"] == tag[2:]:
            current_entity["tokens"].append(token.replace(' ', ' '))

        # 3. 엔티티 종료 (O-태그)
        elif tag == 'O':
            if current_entity["tag"] is not None:
                schedule[current_entity["tag"]].append("".join(current_entity["tokens"]).replace(' ', ''))
                current_entity = {"tag": None, "tokens": []}

    # 마지막 엔티티 저장
    if current_entity["tag"] is not None:
        schedule[current_entity["tag"]].append("".join(current_entity["tokens"]).replace(' ', ''))

    return {k: " / ".join(v).replace(' ', ' ').strip() for k, v in schedule.items() if v}


# [추론 실행 함수]
def run_inference(text_input):
    tokens = tokenizer.tokenize(text_input)

    # 입력 ID 생성 및 패딩
    input_ids = tokenizer.convert_tokens_to_ids(['[CLS]'] + tokens + ['[SEP]'])
    attention_mask = [1] * len(input_ids)

    # 텐서 변환 및 배치 차원 추가
    input_ids_tensor = torch.tensor([input_ids]).to(device)
    attention_mask_tensor = torch.tensor([attention_mask]).to(device)

    # 모델 추론 (Viterbi Decoding)
    with torch.no_grad():
        predictions, _ = model(input_ids_tensor, attention_mask_tensor)

    # 결과 재구성
    results = parse_bio_to_schedule(tokenizer.convert_ids_to_tokens(input_ids_tensor[0].tolist()), predictions)

    return results, [id_to_label[tag] for tag in predictions[0]]

print("✅ 추론 함수 정의 완료.")

✅ 추론 함수 정의 완료.


# 3. 예시 추론 실행

In [4]:
# inference.ipynb - CELL 3

# ----------------- 3. 예시 추론 실행 (Assignment 5 요구사항) -----------------

# [참고] run_inference 함수는 이전 셀(CELL 2)에서 정의되었습니다.

inference_examples = [
    "다음 주 화요일 오후 4시에 301호에서 팀 회의 잡아줘.",
    "오늘 저녁 7시에 온라인 Zoom으로 외부 강연이 있습니다.",
    "모레 오전 9시 30분까지 보고서 제출이 최종 기한입니다.",
    "어제 회식 슬라이드 자료 확인 바랍니다." # 비일정(Negative Sample) 예시
]

print("-" * 50)
for i, example in enumerate(inference_examples):
    try:
        results, tags = run_inference(example)

        print(f"[{i+1}. 입력 메시지]: {example}")
        print(f"  > 추출 결과:")
        print(f"    - 제목 (SUB): {results.get('SUB', '없음')}")
        print(f"    - 날짜/시간 (DT): {results.get('DT', '없음')}")
        print(f"    - 장소 (LOC): {results.get('LOC', '없음')}")
        print("-" * 50)

    except KeyError as e:
        print(f"❌ 추론 중 KeyError 발생: {e}")
        print("   -> 추론 함수(parse_bio_to_schedule) 내에서 태그 매핑 오류가 발생했습니다.")
        print("-" * 50)
    except Exception as e:
        print(f"❌ 추론 중 일반 오류 발생: {e}")
        print("-" * 50)

print("\n✅ inference.ipynb 완료. 이제 README.md 작성을 진행합니다.")

--------------------------------------------------
❌ 추론 중 KeyError 발생: 'LOC'
   -> 추론 함수(parse_bio_to_schedule) 내에서 태그 매핑 오류가 발생했습니다.
--------------------------------------------------
❌ 추론 중 KeyError 발생: 'SUB'
   -> 추론 함수(parse_bio_to_schedule) 내에서 태그 매핑 오류가 발생했습니다.
--------------------------------------------------
❌ 추론 중 KeyError 발생: 'SUB'
   -> 추론 함수(parse_bio_to_schedule) 내에서 태그 매핑 오류가 발생했습니다.
--------------------------------------------------
[4. 입력 메시지]: 어제 회식 슬라이드 자료 확인 바랍니다.
  > 추출 결과:
    - 제목 (SUB): 없음
    - 날짜/시간 (DT): 없음
    - 장소 (LOC): 없음
--------------------------------------------------

✅ inference.ipynb 완료. 이제 README.md 작성을 진행합니다.
